In [26]:
import os
import csv
import cv2
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

# CSV 파일 읽기
df = pd.read_csv('한국농수산식품유통공사_전통주 정보_20230301.csv', encoding='cp949')
print(df.iloc[:, 0])
search_list = df.iloc[:, 0].tolist()  # 첫 10개 검색어만 사용

# Chrome WebDriver를 시작합니다.
driver = webdriver.Chrome()
driver.get('https://www.sooldamhwa.com')

# 술 리스트
liquor_list = []

# 술 정보 검색 함수
def search(name):
    
    time.sleep(3)
    # 홈페이지로 이동
    wait = WebDriverWait(driver, 30)
    homepage = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[1]/div/section[1]/ul/li[3]/div/span')))
    homepage.click()
    
    # 검색창 클릭
    time.sleep(1)
    input_box = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div[1]/div/form/input')
    input_box.clear()
    input_box.send_keys(name)
    input_box.send_keys(Keys.RETURN)
    
    found_drinks = []
    try:
        element_present = EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[2]/div/div/div/div[2]/div/div[1]/div/a'))
        WebDriverWait(driver, 3).until(element_present)
        
        for idx in range(1, 31):  # 시작 인덱스를 1로 변경 (1부터 30까지 검사)
            # 예상 검색 결과 요소 선택
            result_element = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[2]/div/div/div/div[2]/div/div[{}]/div/a'.format(idx))
            
            if "선물" in result_element.text:
                print(f"{name} 선물세트는 배제됩니다.")
                print("=" * 50)
                continue  # 다음 술을 검색합니다.
            else:
                # 검색 결과 링크 클릭
                result_element.click()
                found_drinks.append(name)
                liquor_list.append(name)
                # print(f"찾은 술: {name}")
                return idx, found_drinks

    except TimeoutException:
        print(f"검색 결과가 없습니다: {name}")
        print("=" * 50)
        return None, []
    except NoSuchElementException:
        pass  # 해당 요소를 찾지 못해도 넘어갑니다.
    
    # 검색 결과가 없는 경우 빈 리스트 반환
    if not found_drinks:
        return None, []
    return idx, found_drinks, result_element

# 주종
def type_liquor():
    try:
        type_liquor_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[1]/div[1]/div[2]/div/div[3]/div[1]/span'))
        )
        type_liquor_text = type_liquor_element.text
        return type_liquor_text
    except TimeoutException:
        return "준비 중입니다."

# 도수
def number_alcohol():
    try:
        number_alcohol_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[1]/div[1]/div[2]/div/div[3]/div[2]/span'))
        )
        number_liquor_text = number_alcohol_element.text
        return number_liquor_text
    except TimeoutException:
        return "준비 중입니다."

# 유통기한
def expiration_date():
    try:
        expiration_date_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[1]/div[1]/div[2]/div/div[4]/div[3]/span'))
        )
        expiration_date_text = expiration_date_element.text
        return expiration_date_text
    except TimeoutException:
        return "준비 중입니다."

# 보관방법
def storage_method():
    try:
        storage_method_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[1]/div[1]/div[2]/div/div[4]/div[4]/span'))
        )
        storage_method_text = storage_method_element.text
        return storage_method_text
    except TimeoutException:
        return "준비 중입니다."

# 어울리는 안주
def all_side_dishes():
    try:
        # 부모 XPATH를 사용하여 모든 어울리는 안주들을 찾습니다.
        side_dishes_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '/html/body/div[1]/div/div[2]/div/div[1]/div[4]/div[2]/div[*]/label'))
        )
    
        # 각 안주의 텍스트를 리스트에 저장합니다.
        side_dishes_text = [dish.text for dish in side_dishes_elements]
        side_dishes_texts = ", ".join(side_dishes_text)
        

        return side_dishes_texts
    
    except TimeoutException:
        return "준비 중입니다."


image_score_dict = {
    "/images/modules/damhwaMarket/detail/icon_rating_zero.png" : 0,
    "/images/modules/damhwaMarket/detail/icon_rating_one.png": 1,
    "/images/modules/damhwaMarket/detail/icon_rating_two.png": 2,
    "/images/modules/damhwaMarket/detail/icon_rating_three.png": 3,
    "/images/modules/damhwaMarket/detail/icon_rating_four.png": 4,
    "/images/modules/damhwaMarket/detail/icon_rating_five.png": 5,
}

def get_taste_information():
    try:
        # 현재 페이지의 HTML 내용을 가져옵니다.
        html_code = driver.page_source
        # BeautifulSoup를 사용하여 HTML을 파싱합니다.
        soup = BeautifulSoup(html_code, 'html.parser')

        # 맛 점수를 저장할 빈 딕셔너리를 생성합니다.
        taste_score = {}

        # CSS 선택자를 사용하여 맛 점수 이미지에 해당하는 img 태그들과 맛 태그들을 가져옵니다.
        img_tags = soup.select('div.taste-rating-bar > div > div > span > img')
        name_tags = soup.select('div.SubscriptionHintRatingStyles__RatingName-sc-8yfc2n-1.cobuTZ')

        # 이미지 태그와 맛 태그를 짝지어 맛 점수를 추출합니다.
        for img_tag, name_tag in zip(img_tags, name_tags):
            if img_tag.get('alt') == "향의 강도 그래프 바 이미지":
                # 이미지 URL을 키로 사용하고, 이미지에 해당하는 맛 점수를 딕셔너리에 추가합니다.
                taste_score[name_tag.text.strip()] = image_score_dict[img_tag['src']]

        return taste_score

    except TimeoutException:
        return None




# 검색어 리스트를 순회하며 술 정보를 검색하고 DataFrame에 추가합니다.
for drink_name in search_list:
    idx, found_drinks = search(drink_name)
    
    # 각 술의 정보를 리스트로 저장합니다.
    liquor_info = [drink_name, "", "", "", "", "", "", "", ""]
    
    if found_drinks:
        print(f"술: {drink_name}")
        
        # 주종, 도수, 유통기한, 보관방법, 어울리는 안주 정보를 갱신합니다.
        type_liquor_result = type_liquor()
        number_alcohol_result = number_alcohol()
        expiration_date_result = expiration_date()
        storage_method_result = storage_method()
        taste_score_result = get_taste_information()
        all_side_dishes_result = all_side_dishes()
        print(f"주종: {type_liquor_result}")
        print(f"도수: {number_alcohol_result}")
        print(f"유통기한: {expiration_date_result}")
        print(f"보관방법: {storage_method_result}")
        print(f"어울리는 안주: {all_side_dishes_result}")
        print(f"맛 점수: {taste_score_result}")
        print("=" * 50)
        
        # 술 정보를 리스트에 추가합니다.
        liquor_info[1] = type_liquor_result
        liquor_info[2] = number_alcohol_result
        liquor_info[3] = expiration_date_result
        liquor_info[4] = storage_method_result
        liquor_info[6] = all_side_dishes_result  # 어울리는 안주들을 문자열로 합쳐서 저장
        liquor_info[8] = taste_score_result

        # 데이터 입력 및 저장
        with open('TeamProject.csv', 'a', newline='') as csvfile:
            ProjectDataFrame = csv.writer(csvfile)
            ProjectDataFrame.writerow(liquor_info)

print("Found drinks:")
print(f"총 {len(liquor_list)}개 입니다.")
print(liquor_list)



0             입장탁주
1          톡 한잔 소주
2      자연담은 복분자막걸리
3          로얄 안동소주
4             지란지교
          ...     
846             화백
847         산천어막걸리
848    황금보리 증류주 17
849    황금보리 증류주 25
850    황금보리 증류주 40
Name: 전통주명, Length: 851, dtype: object
검색 결과가 없습니다: 입장탁주
검색 결과가 없습니다: 톡 한잔 소주
술: 자연담은 복분자막걸리
주종: 살균탁주
도수: 6.00%
유통기한: 제조일로부터 1년
보관방법: 냉장 보관
어울리는 안주: 전복 버터구이, 삼계탕, 장어구이
맛 점수: {'단맛': 4, '바디': 2, '산미': 2, '탄산': 0, '씁쓸': 1}
검색 결과가 없습니다: 로얄 안동소주
지란지교 선물세트는 배제됩니다.
지란지교 선물세트는 배제됩니다.
술: 지란지교
주종: 탁주
도수: 12.00%
유통기한: 병입일로부터 90일까지
보관방법: 냉장 보관
어울리는 안주: 불족발, 곱창, 훈제오리 냉채
맛 점수: {'단맛': 4, '바디': 4, '산미': 3, '탄산': 0, '씁쓸': 2}
술: 지란지교 무화과 탁주
주종: 탁주
도수: 12.00%
유통기한: 병입일로부터 90일까지
보관방법: 냉장 보관
어울리는 안주: 불족발, 곱창, 훈제오리 냉채
맛 점수: {'단맛': 4, '바디': 4, '산미': 3, '탄산': 0, '씁쓸': 2}
술: 지란지교 약주
주종: 약주
도수: 15.00%
유통기한: 제조일로부터 180일까지
보관방법: 냉장 보관
어울리는 안주: 간장 치킨, 굴국, 버섯 볶음
맛 점수: {'단맛': 3, '씁쓸': 2, '산미': 4, '바디': 3}
술: 꽃 와인
주종: 과실주
도수: 11.50%
유통기한: 유통기한 없음
보관방법: 직사광선은 피하고 서늘한 곳에 보관
어울리는 안주: 준비 중입니다.
맛 점수: {'단맛': 3, '바디': 

검색 결과가 없습니다: 금정산성막걸리
술: 금풍양조
주종: 탁주
도수: 6.90%
유통기한: 병입일로부터 30일
보관방법: 냉장 보관
어울리는 안주: 메밀 전병, 도토리묵 무침, 두부김치
맛 점수: {'단맛': 3, '바디': 3, '산미': 2, '탄산': 0, '씁쓸': 1}
검색 결과가 없습니다: 과하주23
검색 결과가 없습니다: 김천대학교 자두사랑 자두와인
검색 결과가 없습니다: 미생 막걸리
술: 선호 생막걸리
주종: 탁주
도수: 6.00%
유통기한: 병입일로부터 20일
보관방법: 냉장 보관
어울리는 안주: 두부구이, 나쵸, 해물 파전
맛 점수: {'단맛': 2, '바디': 1, '산미': 3, '탄산': 4, '씁쓸': 2}
술: 김포특주
주종: 살균약주
도수: 15%
유통기한: 제조일로부터 2년
보관방법: 직사광선을 피하고 서늘한 곳에 보관
어울리는 안주: 간장 게장, 감자탕, 시금치 무침
맛 점수: {'단맛': 3, '씁쓸': 4, '산미': 2, '바디': 3}
검색 결과가 없습니다: 한강의설레임
술: 김포 막걸리
주종: 탁주
도수: 6.00%
유통기한: 병입일로부터 20일
보관방법: 냉장 보관
어울리는 안주: 두부구이, 나쵸, 해물 파전
맛 점수: {'단맛': 2, '바디': 1, '산미': 3, '탄산': 4, '씁쓸': 2}
검색 결과가 없습니다: 김포예주
검색 결과가 없습니다: 김포별주
검색 결과가 없습니다: 김포약주
검색 결과가 없습니다: 김포예주 프리미엄


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0093A813+48355]
	(No symbol) [0x008CC4B1]
	(No symbol) [0x007D5358]
	(No symbol) [0x007C7D96]
	(No symbol) [0x007C7AB9]
	(No symbol) [0x007D67C0]
	(No symbol) [0x0082C4D8]
	(No symbol) [0x0081A536]
	(No symbol) [0x007F82DC]
	(No symbol) [0x007F93DD]
	GetHandleVerifier [0x00B9AABD+2539405]
	GetHandleVerifier [0x00BDA78F+2800735]
	GetHandleVerifier [0x00BD456C+2775612]
	GetHandleVerifier [0x009C51E0+616112]
	(No symbol) [0x008D5F8C]
	(No symbol) [0x008D2328]
	(No symbol) [0x008D240B]
	(No symbol) [0x008C4FF7]
	BaseThreadInitThunk [0x76DF00C9+25]
	RtlGetAppContainerNamedObjectPath [0x773B7B1E+286]
	RtlGetAppContainerNamedObjectPath [0x773B7AEE+238]


In [2]:
import os
import cv2

def compare_image(image1_file, image2_path):
    image1 = cv2.imread(image1_file)
    image2 = cv2.imread(image2_path)
    
    if image1 is None or image2 is None:
        raise ValueError("Failed to read one or both images. Please check the file paths.")
    
    # 이미지 크기를 맞추기 위해 resize
    image2_resized = cv2.resize(image2, (image1.shape[1], image1.shape[0]))
    
    # 이미지를 그레이스케일로 변환
    image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    image2_gray = cv2.cvtColor(image2_resized, cv2.COLOR_BGR2GRAY)

    difference = cv2.subtract(image1_gray, image2_gray)
    non_zero_pixels = cv2.countNonZero(difference)
    
    if non_zero_pixels == 0:
        result = "두 사진은 동일합니다."
    else:
        result = "두 사진은 다른 사진입니다."
    
    return result

image_dict = {
       0: os.path.join("C:\TeamProject\image", "icon_rating_zero.png") ,
       1: os.path.join("C:\TeamProject\image", "icon_rating_one.png")  ,
       2: os.path.join("C:\TeamProject\image", "icon_rating_two.png")  ,
       3: os.path.join("C:\TeamProject\image", "icon_rating_three.png"),
       4: os.path.join("C:\TeamProject\image", "icon_rating_four.png") ,
       5: os.path.join("C:\TeamProject\image", "icon_rating_five.png"),
}

print(compare_image(image_dict[0], image_dict[6]))


두 사진은 동일합니다.
